In [1]:

import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.prompts import PromptTemplate
from langchain import hub

In [3]:

template = """
Please describe in detail the contents of the book “{title}”. Please provide as much specific information as possible about what the book covers, important themes, genre, and a summary of the content.
Do not include any information unrelated to the content, such as the size of the book.
"""

prompt = PromptTemplate(template=template, input_variables=["title"])
prompt_format = prompt.format(title="時々ボソッとロシア語でデレるアーリャさん")
print(prompt_format)



Please describe in detail the contents of the book “時々ボソッとロシア語でデレるアーリャさん”. Please provide as much specific information as possible about what the book covers, important themes, genre, and a summary of the content.
Do not include any information unrelated to the content, such as the size of the book.



In [4]:
tools = ["google-search"]
llm = ChatOpenAI(model="gpt-3.5-turbo")
tools = load_tools(tools, llm=llm)
agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True)

/Users/kai-yamashita/Desktop/langchain/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [5]:
output = agent.invoke(prompt_format)
print(output)



> Entering new AgentExecutor chain...
I should use Google Search to find information about the book "時々ボソッとロシア語でデレるアーリャさん".
Action: google_search
Action Input: "時々ボソッとロシア語でデレるアーリャさん book summary"
Observation: シリーズ累計発行部数170万部超！ 大人気ライトノベル・コミックが待望のTVアニメ化. 商品の詳細 · 商品の説明 · クチコミ評価 · 最新のクチコミ · 本・音楽・ゲーム・文学/小説・本・音楽・ゲーム・文学/小説 · この商品を見ている人におすすめ. 商品の状態目立った傷や汚れなし配送料の負担送料込み(出品者負担)配送の方法らくらくメルカリ便発送元の地域福岡県発送までの日数4~7日で発送 ... 時々ボソッとロシア語でデレる隣のアーリャさん アクリルスタンド 第5 ... グッズ-スタンドポップ】時々ボソッとロシア語 ... 燦々SUN先生×ももこ先生のタッグで贈るニヤニヤ必至な青春ラブコメ！『時々ボソッとロシア語でデレる隣のアーリャさん』公式アカウント。 公式タグ #ロシデレ あらすじ→ ... 新品未開封品です。プチプチ梱包して発送致します。・時々ボソッとロシア語でデレる隣のアーリャさん B2タペストリー/第3弾 A・ゲーマーズ「超」限定版 B2 ... 大特価✨時々ボソッとロシア語でデレる隣のアーリャさんロシデレ アーリャさんアクリルスタンド21点セット✨新品ではありません。飾っていました。 以前サンプルを使わせていただいてまつ毛がのびたのでさっそくリピ購入です。他のまつ毛美容液も数種類試しましたが、効果はあるものの色素沈着がひどいので、こちらの ... 7巻あらすじ. 戸惑いとトキメキが交錯した体育祭を経て、遂に自分の気持ちに気付いたアーリャさん。一方、政近は約束の ... 【全巻セット】時々ボソッとロシア語でデレる隣のアーリャさん 1巻 ～ 3巻 (講談社コミックス) - 漫画、コミック.
Thought:I should try a more specific search to find a summary of the book.


In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
query_result = embeddings.embed_query(output["output"])
print(len(query_result))
print(query_result)

In [1]:
import asyncio
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
import torch

tools = ["google-search"]
llm = ChatOpenAI(model="gpt-3.5-turbo")
tools = load_tools(tools, llm=llm)
agent = initialize_agent(tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True)

template = """
「{title}」の本の内容を詳細に説明してください．その本ではどのような内容が扱われているのか，重要なテーマ，ジャンル，内容の要約について，具体的な情報をできるだけ多く教えてください．
"""
preference_temlate = """
ユーザーの要求：「{preference}」
このユーザーの読みたい本の要求に基づいて，ユーザーにおすすめするべき本は，どのような内容で，テーマ，ジャンルであるべきか，具体的な情報を挙げなさい
"""

prompt = PromptTemplate(template=template, input_variables=["title"])
preference_prompt = PromptTemplate(template=preference_temlate, input_variables=["preference"])

async def get_content(title):
    book_prompt = prompt.format(title=title)
    output = await agent.ainvoke(book_prompt)
    print(output["output"])
    return output

async def get_books_content(books):
    tasks = [get_content(title) for title in books]
    return await asyncio.gather(*tasks)


books = ["鋼の錬金術師", "人間失格", "容疑者Xの献身", "解析入門1", "人工知能は人間を超えるか", "時々ボソッとロシア語でデレる隣のアーリャさん"]

results = await get_books_content(books)
embeddings = OpenAIEmbeddings()
query_result = embeddings.embed_documents([result["output"] for result in results])

book_data = torch.tensor(query_result)
print(book_data.shape)

/Users/kai-yamashita/Desktop/langchain/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

> Entering new AgentExecutor chain...



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...
It seems like the question is asking for a detailed explanation of the book "The Devotion of Suspect X" in Japanese.
Action: google_search
Action Input: "The Devotion of Suspect X book summary"I should use the google_search tool to find detailed information about the book "人間失格".
Action: google_search
Action Input: "人間失格 本 内容"

: 

In [ ]:
preferences = llm(preference_prompt.format(preference="ラブコメ作品が読みたい"))
preference_emb = embeddings.embed_query(preferences["output"])

pfr = torch.tensor(preference_emb)
pfr = pfr.repeat(book_data.shape[0], 1)
similarity = torch.nn.functional.cosine_similarity(book_data, pfr, dim=1)

max_index = torch.argmax(similarity)
print(books[max_index.item()])




In [6]:
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.prompts import PromptTemplate
from langchain import hub
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo")
message = [
    ("system", "You are a dictionary that provides information about genres and contents of books which are requested by users. Do not include any specific titles of books in your response. "),
    ("human", "ラプコメが読みたい")]
fuge = llm.invoke(message)
print(fuge.content)

ラプコメ（ライトノベルとコメディの略）は、ライトノベルの要素とコメディ要素が組み合わさったジャンルの小説です。一般的には、軽快で笑いを取り入れたストーリー展開やキャラクターのやり取りが特徴とされています。主人公がコメディ的な状況に巻き込まれたり、ユーモアを交えながら冒険や日常生活を描いた作品が含まれます。
